In [ ]:
import pandas as pd

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Saturdays/lichess_db_puzzle.csv')

In [ ]:
print(df.columns)

Index(['PuzzleId', 'FEN', 'Moves', 'Rating', 'RatingDeviation', 'Popularity',
       'NbPlays', 'Themes', 'GameUrl', 'OpeningTags'],
      dtype='object')


In [ ]:
df = df[['FEN', 'Themes']]
print(df)

                                                       FEN  \
0        r6k/pp2r2p/4Rp1Q/3p4/8/1N1P2R1/PqP2bPP/7K b - ...   
1        5rk1/1p3ppp/pq3b2/8/8/1P1Q1N2/P4PPP/3R2K1 w - ...   
2        r2qr1k1/b1p2ppp/pp4n1/P1P1p3/4P1n1/B2P2Pb/3NBP...   
3                   8/8/4k1p1/2KpP2p/5PP1/8/8/8 w - - 0 53   
4        4r3/1k6/pp3r2/1b2P2p/3R1p2/P1R2P2/1P4PP/6K1 w ...   
...                                                    ...   
3251243  3r1rk1/3pN1pp/nq3pn1/1pp1p1QN/4P3/5P2/2P3PP/3R...   
3251244   3r3k/p5pp/8/5R2/1BQ1p3/P3q3/Bb4PP/6K1 w - - 0 28   
3251245  5Q2/pp3R1P/1kpp4/4p3/2P1P3/3PP2P/Pr2q3/2K5 w -...   
3251246  r3kb1r/ppp2ppp/2n5/3q3b/3P1B2/5N1P/PPP3P1/RN1Q...   
3251247  r2q1rk1/4bppp/p1n1pn2/1p1pN3/2pP2b1/1PP1P3/PBQ...   

                                             Themes  
0             crushing hangingPiece long middlegame  
1                           advantage endgame short  
2                        advantage middlegame short  
3                 crushing endgame long

In [ ]:
import re
import numpy as np

#All themes. If themes deleted or merged, process_themes and process_fen must be adapted
all_themes = [
    'advantage', 'hangingPiece', 'bodenMate', 'clearance', 'endgame', 'opening', 'defensiveMove',
    'oneMove', 'attackingF2F7', 'enPassant', 'kingsideAttack', 'castling', 'pawnEndgame', 'deflection', 'arabianMate',
    'interference', 'queenRookEndgame', 'mateIn5', 'smotheredMate', 'exposedKing', 'crushing', 'dovetailMate', 'hookMate',
    'knightEndgame', 'mateIn1', 'promotion', 'pin', 'masterVsMaster', 'backRankMate', 'attraction', 'equality',
    'veryLong', 'queensideAttack', 'bishopEndgame', 'middlegame', 'underPromotion', 'doubleCheck', 'queenEndgame', 'rookEndgame',
    'skewer', 'capturingDefender', 'superGM', 'zugzwang', 'long', 'anastasiaMate', 'intermezzo', 'short',
    'mateIn2', 'xRayAttack', 'discoveredAttack', 'master', 'mateIn3', 'mateIn4', 'mate', 'trappedPiece',
    'doubleBishopMate', 'fork', 'advancedPawn', 'quietMove', 'sacrifice']

# Mapping for pieces
piece_mapping = {'p':1, 'n':2, 'b':3, 'r':4, 'q':5, 'k':6, ' ':7,
                 'P':8, 'N':9, 'B':10,'R':11,'Q':12,'K':13}

# Mapping for en passant
square_mapping = {
    '-':0,
    'a8': 1, 'b8': 2, 'c8': 3, 'd8': 4, 'e8': 5, 'f8': 6, 'g8': 7, 'h8': 8,
    'a7': 9, 'b7': 10, 'c7': 11, 'd7': 12, 'e7': 13, 'f7': 14, 'g7': 15, 'h7': 16,
    'a6': 17, 'b6': 18, 'c6': 19, 'd6': 20, 'e6': 21, 'f6': 22, 'g6': 23, 'h6': 24,
    'a5': 25, 'b5': 26, 'c5': 27, 'd5': 28, 'e5': 29, 'f5': 30, 'g5': 31, 'h5': 32,
    'a4': 33, 'b4': 34, 'c4': 35, 'd4': 36, 'e4': 37, 'f4': 38, 'g4': 39, 'h4': 40,
    'a3': 41, 'b3': 42, 'c3': 43, 'd3': 44, 'e3': 45, 'f3': 46, 'g3': 47, 'h3': 48,
    'a2': 49, 'b2': 50, 'c2': 51, 'd2': 52, 'e2': 53, 'f2': 54, 'g2': 55, 'h2': 56,
    'a1': 57, 'b1': 58, 'c1': 59, 'd1': 60, 'e1': 61, 'f1': 62, 'g1': 63, 'h1': 64
}

def process_fen(fen_string):
    # Splitting FEN string into parts
    board, turn, castling, en_passant, _ , _ = fen_string.split()

    # Transforming board
    board = board.replace("/", "")  # removing slash
    for digit in "12345678":
        board = board.replace(digit, " " * int(digit))  # replace numbers with spaces
    board_values = [piece_mapping[piece] for piece in board]  # map pieces to numbers

    # Transforming metadata
    metadata = [1 if turn == 'b' else 0]
    metadata.extend([1 if c in castling else 0 for c in 'KQkq'])
    # Handle the case when there's no valid en passant target square
    metadata.append(square_mapping[en_passant] if en_passant in square_mapping else 0)
    metadata.extend([0, 0])  # two empty squares as per the requirement

    # Adding the board and metadata together
    fen_matrix = np.array(board_values + metadata).reshape((9, 8))

    return fen_matrix

#Generate the embedding vector for the themes
def process_themes(themes_string):
    return [1 if theme in themes_string else 0 for theme in all_themes]

#Transform FEN to 9x8 array where 1x8 is for metadata like castling, turn and en passant
def process_df_row(row):
    fen_matrix = process_fen(row['FEN'])
    themes = np.array(process_themes(row['Themes']))
    return fen_matrix, themes

#Process df and generate X and Y up to n rows
def generate_X_Y(df, n):
    # Apply processing functions to the first `n` rows of the dataframe
    results = df.iloc[:n].apply(process_df_row, axis=1)

    # Stack the resulting numpy arrays into a 3D array for X and a 2D array for Y
    X = np.stack(results.apply(lambda x: x[0]))
    Y = np.stack(results.apply(lambda x: x[1]))

    return X, Y

In [ ]:
len(all_themes)

60

In [ ]:
fen_string = 'rnbqkbnr/pppp1ppp/8/4p3/3P4/8/PPP1PPPP/RNBQKBNR w Kkq h2 0 1'
themes_string = 'hangingPiece advancedPawn'

processed_fen = process_fen(fen_string)
print(processed_fen)

processed_themes = process_themes(themes_string)
print(processed_themes)

[[ 4  2  3  5  6  3  2  4]
 [ 1  1  1  1  7  1  1  1]
 [ 7  7  7  7  7  7  7  7]
 [ 7  7  7  7  1  7  7  7]
 [ 7  7  7  8  7  7  7  7]
 [ 7  7  7  7  7  7  7  7]
 [ 8  8  8  7  8  8  8  8]
 [11  9 10 12 13 10  9 11]
 [ 0  1  0  1  1 56  0  0]]
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]


In [ ]:
X, Y = generate_X_Y(df, 10)

X, Y

(array([[[ 4,  7,  7,  7,  7,  7,  7,  6],
         [ 1,  1,  7,  7,  4,  7,  7,  1],
         [ 7,  7,  7,  7, 11,  1,  7, 12],
         [ 7,  7,  7,  1,  7,  7,  7,  7],
         [ 7,  7,  7,  7,  7,  7,  7,  7],
         [ 7,  9,  7,  8,  7,  7, 11,  7],
         [ 8,  5,  8,  7,  7,  3,  8,  8],
         [ 7,  7,  7,  7,  7,  7,  7, 13],
         [ 1,  0,  0,  0,  0,  0,  0,  0]],
 
        [[ 7,  7,  7,  7,  7,  4,  6,  7],
         [ 7,  1,  7,  7,  7,  1,  1,  1],
         [ 1,  5,  7,  7,  7,  3,  7,  7],
         [ 7,  7,  7,  7,  7,  7,  7,  7],
         [ 7,  7,  7,  7,  7,  7,  7,  7],
         [ 7,  8,  7, 12,  7,  9,  7,  7],
         [ 8,  7,  7,  7,  7,  8,  8,  8],
         [ 7,  7,  7, 11,  7,  7, 13,  7],
         [ 0,  0,  0,  0,  0,  0,  0,  0]],
 
        [[ 4,  7,  7,  5,  4,  7,  6,  7],
         [ 3,  7,  1,  7,  7,  1,  1,  1],
         [ 1,  1,  7,  7,  7,  7,  2,  7],
         [ 8,  7,  8,  7,  1,  7,  7,  7],
         [ 7,  7,  7,  7,  8,  7,  2,  7],
     

In [ ]:
df[:10]

,FEN,Themes
0,r6k/pp2r2p/4Rp1Q/3p4/8/1N1P2R1/PqP2bPP/7K b - ...,crushing hangingPiece long middlegame
1,5rk1/1p3ppp/pq3b2/8/8/1P1Q1N2/P4PPP/3R2K1 w - ...,advantage endgame short
2,r2qr1k1/b1p2ppp/pp4n1/P1P1p3/4P1n1/B2P2Pb/3NBP...,advantage middlegame short
3,8/8/4k1p1/2KpP2p/5PP1/8/8/8 w - - 0 53,crushing endgame long pawnEndgame
4,4r3/1k6/pp3r2/1b2P2p/3R1p2/P1R2P2/1P4PP/6K1 w ...,endgame mate mateIn2 short
5,r4rk1/pp3ppp/2n1b3/q1pp2B1/8/P1Q2NP1/1PP1PP1P/...,advantage master middlegame short
6,r1bqk2r/pp1nbNp1/2p1p2p/8/2BP4/1PN3P1/P3QP1P/3...,mate mateIn2 middlegame short
7,5r1k/5rp1/p7/1b2B2p/1P1P1Pq1/2R1Q3/P3p1P1/2R3K...,crushing middlegame short
8,3R4/8/K7/pB2b3/1p6/1P2k3/3p4/8 w - - 4 58,crushing endgame fork short
9,4r3/5pk1/1p3np1/3p3p/2qQ4/P4N1P/1P3RP1/7K w - ...,crushing endgame short trappedPiece


In [ ]:
df2 = pd.read_csv('/content/drive/MyDrive/Saturdays/merged_sequences.csv')

df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 631278 entries, 0 to 631277
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  631278 non-null  int64 
 1   SEQUENCE    631278 non-null  object
 2   EVAL        631278 non-null  object
 3   Themes      631278 non-null  object
dtypes: int64(1), object(3)
memory usage: 19.3+ MB


In [ ]:
def filter_eval(df):
    # Filter rows meeting the condition EVAL > 200 and EVAL < -200
    df['EVAL'] = pd.to_numeric(df['EVAL'], errors='coerce')   # Convertir los valores de la columna 'EVAL' a numéricos
    filtro = (df['EVAL'] > 200) | (df['EVAL'] < -200)
    df_filtered = df[filtro]
    return df_filtered

In [ ]:
df2 = filter_eval(df2)

In [ ]:
import re

fen_string = "r2q1rk1/1pp3p1/p4n1p/4p3/1P1n4/P1NP1N1P/2P1Q1P1/R4RK1 w - - 0 16 r2q1rk1/1pp3p1/p4n1p/4Q3/1P1n4/P1NP1N1P/2P3P1/R4RK1 b - - 0 16 r2q1rk1/1pp3p1/p4n1p/4Q3/1P6/P1NP1N1P/2n3P1/R4RK1 w - - 0 17 r2q1rk1/1pp3p1/p4n1p/4Q3/1P6/P1NP1N1P/2n3P1/2R2RK1 b - - 1 17 r4rk1/1pp3p1/p4n1p/4Q3/1P6/P1Nq1N1P/2n3P1/2R2RK1 w - - 0 18 r4rk1/1pp3p1/p4n1p/4Q3/1P6/P2q1N1P/2n3P1/2RN1RK1 b - - 1 18 4rrk1/1pp3p1/p4n1p/4Q3/1P6/P2q1N1P/2n3P1/2RN1RK1 w - - 2 19 4rrk1/1pQ3p1/p4n1p/8/1P6/P2q1N1P/2n3P1/2RN1RK1 b - - 0 19 2r2rk1/1pQ3p1/p4n1p/8/1P6/P2q1N1P/2n3P1/2RN1RK1 w - - 1 20 2r2rk1/1Q4p1/p4n1p/8/1P6/P2q1N1P/2n3P1/2RN1RK1 b - - 0 20"

fen_list = re.split(r"\s(?=\w+/)", fen_string)

print(fen_list)


['r2q1rk1/1pp3p1/p4n1p/4p3/1P1n4/P1NP1N1P/2P1Q1P1/R4RK1 w - - 0 16', 'r2q1rk1/1pp3p1/p4n1p/4Q3/1P1n4/P1NP1N1P/2P3P1/R4RK1 b - - 0 16', 'r2q1rk1/1pp3p1/p4n1p/4Q3/1P6/P1NP1N1P/2n3P1/R4RK1 w - - 0 17', 'r2q1rk1/1pp3p1/p4n1p/4Q3/1P6/P1NP1N1P/2n3P1/2R2RK1 b - - 1 17', 'r4rk1/1pp3p1/p4n1p/4Q3/1P6/P1Nq1N1P/2n3P1/2R2RK1 w - - 0 18', 'r4rk1/1pp3p1/p4n1p/4Q3/1P6/P2q1N1P/2n3P1/2RN1RK1 b - - 1 18', '4rrk1/1pp3p1/p4n1p/4Q3/1P6/P2q1N1P/2n3P1/2RN1RK1 w - - 2 19', '4rrk1/1pQ3p1/p4n1p/8/1P6/P2q1N1P/2n3P1/2RN1RK1 b - - 0 19', '2r2rk1/1pQ3p1/p4n1p/8/1P6/P2q1N1P/2n3P1/2RN1RK1 w - - 1 20', '2r2rk1/1Q4p1/p4n1p/8/1P6/P2q1N1P/2n3P1/2RN1RK1 b - - 0 20']


In [ ]:
def process_eval(array, eval):
    # Insert an additional dimension into the array
    array_with_number= np.expand_dims(array, axis=3)
    # Assign the numerical value to each array
    array_with_number[:] = eval

    return array_with_number

In [ ]:
#All themes. If themes deleted or merged, process_themes and process_fen must be adapted
all_themes = [
    'advantage', 'hangingPiece', 'bodenMate', 'clearance', 'endgame', 'opening', 'defensiveMove',
    'oneMove', 'attackingF2F7', 'enPassant', 'kingsideAttack', 'castling', 'pawnEndgame', 'deflection', 'arabianMate',
    'interference', 'queenRookEndgame', 'mateIn5', 'smotheredMate', 'exposedKing', 'crushing', 'dovetailMate', 'hookMate',
    'knightEndgame', 'mateIn1', 'promotion', 'pin', 'masterVsMaster', 'backRankMate', 'attraction', 'equality',
    'veryLong', 'queensideAttack', 'bishopEndgame', 'middlegame', 'underPromotion', 'doubleCheck', 'queenEndgame', 'rookEndgame',
    'skewer', 'capturingDefender', 'superGM', 'zugzwang', 'long', 'anastasiaMate', 'intermezzo', 'short',
    'mateIn2', 'xRayAttack', 'discoveredAttack', 'master', 'mateIn3', 'mateIn4', 'mate', 'trappedPiece',
    'doubleBishopMate', 'fork', 'advancedPawn', 'quietMove', 'sacrifice']

# Mapping for pieces
piece_mapping = {'p':1, 'n':2, 'b':3, 'r':4, 'q':5, 'k':6, ' ':7,
                 'P':8, 'N':9, 'B':10,'R':11,'Q':12,'K':13}

# Mapping for en passant
square_mapping = {
    '-':0,
    'a8': 1, 'b8': 2, 'c8': 3, 'd8': 4, 'e8': 5, 'f8': 6, 'g8': 7, 'h8': 8,
    'a7': 9, 'b7': 10, 'c7': 11, 'd7': 12, 'e7': 13, 'f7': 14, 'g7': 15, 'h7': 16,
    'a6': 17, 'b6': 18, 'c6': 19, 'd6': 20, 'e6': 21, 'f6': 22, 'g6': 23, 'h6': 24,
    'a5': 25, 'b5': 26, 'c5': 27, 'd5': 28, 'e5': 29, 'f5': 30, 'g5': 31, 'h5': 32,
    'a4': 33, 'b4': 34, 'c4': 35, 'd4': 36, 'e4': 37, 'f4': 38, 'g4': 39, 'h4': 40,
    'a3': 41, 'b3': 42, 'c3': 43, 'd3': 44, 'e3': 45, 'f3': 46, 'g3': 47, 'h3': 48,
    'a2': 49, 'b2': 50, 'c2': 51, 'd2': 52, 'e2': 53, 'f2': 54, 'g2': 55, 'h2': 56,
    'a1': 57, 'b1': 58, 'c1': 59, 'd1': 60, 'e1': 61, 'f1': 62, 'g1': 63, 'h1': 64
}

def process_FENs(fens_string, eval):
    #print(fens_string)
    # Splitting FENs secuence in FEN individual
    fens_string = re.split(r"\s(?=\w+/)", fens_string)
    fen_matrices = []
    for fen_string in fens_string:
        #print(fen_string)
        # Splitting FEN string into parts
        board, turn, castling, en_passant, _ , _ = fen_string.split()

        # Transforming board
        board = board.replace("/", "")  # removing slash
        for digit in "12345678":
            board = board.replace(digit, " " * int(digit))  # replace numbers with spaces
        board_values = [piece_mapping[piece] for piece in board]  # map pieces to numbers

        # Transforming metadata
        metadata = [1 if turn == 'b' else 0]
        metadata.extend([1 if c in castling else 0 for c in 'KQkq'])
        # Handle the case when there's no valid en passant target square
        metadata.append(square_mapping[en_passant] if en_passant in square_mapping else 0)
        metadata.extend([0, 0])  # two empty squares as per the requirement

        #print(np.array(board_values + metadata).shape)
        # Adding the board and metadata together
        fen_matrix = np.array(board_values + metadata).reshape((9, 8))
        # Append the fen matrix to the list
        fen_matrices.append(fen_matrix)

    # Check if the array has less than 10 elements
    if len(fen_matrices) < 10:
        # Calculate the amount of elements to fill
        elements_restants = 10 - len(fen_matrices)
        # Filling the array with zeros
        fen_matrices = fen_matrices + [np.zeros((9, 8))] * elements_restants

    # Convert the matrix to a array of form (10, 9, 8)
    array_final = np.array(fen_matrices).reshape((10, 9, 8))
    # Process EVAL number
    array_final = process_eval(array_final, eval)

    return array_final

In [ ]:
#Generate the embedding vector for the themes
def process_themes(themes_string):
    return [1 if theme in themes_string else 0 for theme in all_themes]

In [ ]:
def process_features(row):
    # Get features transformed and scaled
    fen_matrix = process_FENs(row['SEQUENCE'], row['EVAL'])   #Get transform FENs
    themes = np.array(process_themes(row['Themes']))
    return fen_matrix, themes

In [ ]:
#Process df and generate X and Y up to n rows
def generate2_X_Y(df, n):
    # Apply processing functions to the first `n` rows of the dataframe
    results = df.iloc[:n].apply(process_features, axis=1)

    # Stack the resulting numpy arrays into a 4D array for X and a 2D array for Y
    X = np.stack(results.apply(lambda x: x[0]))
    Y = np.stack(results.apply(lambda x: x[1]))

    return X, Y, results

In [ ]:
X, Y, results = generate2_X_Y(df2, len(df2))

#X = np.load('/content/drive/MyDrive/Saturdays/X_5D.npy')
#Y = np.stack(df2['Themes'].iloc[:len(df2)].apply(process_themes))


In [ ]:
# Guardar el arreglo en un archivo
#np.save('X_4D.npy', X)

In [ ]:
X.shape

(248548, 10, 9, 8, 1)

In [ ]:
Y.shape

(248548, 60)

In [ ]:
df2['EVAL']

0         392.0
2         294.0
6        -859.0
8         679.0
10        392.0
          ...  
631267    218.0
631269    280.0
631270    277.0
631274    808.0
631275    410.0
Name: EVAL, Length: 248548, dtype: float64

In [ ]:
X[:, 4]

array([[[[ 392.],
         [ 392.],
         [ 392.],
         ...,
         [ 392.],
         [ 392.],
         [ 392.]],

        [[ 392.],
         [ 392.],
         [ 392.],
         ...,
         [ 392.],
         [ 392.],
         [ 392.]],

        [[ 392.],
         [ 392.],
         [ 392.],
         ...,
         [ 392.],
         [ 392.],
         [ 392.]],

        ...,

        [[ 392.],
         [ 392.],
         [ 392.],
         ...,
         [ 392.],
         [ 392.],
         [ 392.]],

        [[ 392.],
         [ 392.],
         [ 392.],
         ...,
         [ 392.],
         [ 392.],
         [ 392.]],

        [[ 392.],
         [ 392.],
         [ 392.],
         ...,
         [ 392.],
         [ 392.],
         [ 392.]]],


       [[[ 294.],
         [ 294.],
         [ 294.],
         ...,
         [ 294.],
         [ 294.],
         [ 294.]],

        [[ 294.],
         [ 294.],
         [ 294.],
         ...,
         [ 294.],
         [ 294.],
         [ 2

In [ ]:
from sklearn.preprocessing import StandardScaler

# Copiar el arreglo a una nueva variable
copia_X = np.copy(X)

# Escalador para cada dimensión
scaled = StandardScaler()

# Escalar la primera dimensión (columna 4)
X[:, 4] = scaled.fit_transform(X[:, 4].reshape(-1, 1)).flatten().reshape(248548, 9, 8, 1)


print(X[:, 4])


[[[[-0.06659188]
   [-0.06659188]
   [-0.06659188]
   ...
   [-0.06659188]
   [-0.06659188]
   [-0.06659188]]

  [[-0.06659188]
   [-0.06659188]
   [-0.06659188]
   ...
   [-0.06659188]
   [-0.06659188]
   [-0.06659188]]

  [[-0.06659188]
   [-0.06659188]
   [-0.06659188]
   ...
   [-0.06659188]
   [-0.06659188]
   [-0.06659188]]

  ...

  [[-0.06659188]
   [-0.06659188]
   [-0.06659188]
   ...
   [-0.06659188]
   [-0.06659188]
   [-0.06659188]]

  [[-0.06659188]
   [-0.06659188]
   [-0.06659188]
   ...
   [-0.06659188]
   [-0.06659188]
   [-0.06659188]]

  [[-0.06659188]
   [-0.06659188]
   [-0.06659188]
   ...
   [-0.06659188]
   [-0.06659188]
   [-0.06659188]]]


 [[[-0.18013534]
   [-0.18013534]
   [-0.18013534]
   ...
   [-0.18013534]
   [-0.18013534]
   [-0.18013534]]

  [[-0.18013534]
   [-0.18013534]
   [-0.18013534]
   ...
   [-0.18013534]
   [-0.18013534]
   [-0.18013534]]

  [[-0.18013534]
   [-0.18013534]
   [-0.18013534]
   ...
   [-0.18013534]
   [-0.18013534]
   [-0.1801

# Trainnig

### Splitting data

In [ ]:
#Aplanar los datos
X_flat = X.reshape(X.shape[0], -1)

In [ ]:
from sklearn.model_selection import train_test_split

# Dividir los datos en entrenamiento, validación y prueba
X_train, X_val, Y_train, Y_val = train_test_split(X_flat, Y, test_size=0.2, random_state=42)
X_val, X_test, Y_val, Y_test = train_test_split(X_val, Y_val, test_size=0.5, random_state=42)


### Training of the LGBM model using the "one-vs-all" approach

In [ ]:
# Import required libraries
import lightgbm as lgbm
from sklearn.model_selection import train_test_split
import joblib

In [ ]:
# Splitting the data in train and test
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


In [ ]:
# Parameters of the model
params = {
    'objective': 'multiclass',
    'num_class': 60,
    'num_leaves': 31,
    'learning_rate': 0.1,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

In [ ]:
# Reshaping training data to maintain information between dimensions
X_train_flat = X_train.reshape(X_train.shape[0], -1)
X_test_flat = X_test.reshape(X_test.shape[0], -1)

# Create dataset of LightGBM
train_data = lgbm.Dataset(X_train_flat, label=y_train)

In [ ]:
# LGBM model training with early stopping and saving progress
model_path = 'lgbm_model.pkl'
start_round = 0

try:
    # Try to load the previously trained model
    model = joblib.load(model_path)
    start_round = model.best_iteration + 1
    print("Loading previous model from the round:", start_round)
except FileNotFoundError:
    # If the model does not exist, start a new one
    model = None

evals_result = {} # Dictionary for storing evaluation results

if model is None:
    callbacks = [
        lgbm.log_evaluation(period=10)  # Registrar la evaluación cada 10 iteraciones
    ]
    model = lgbm.train(
        params,
        train_data,
        num_boost_round=1000,
        valid_sets=[train_data],
        early_stopping_rounds=None,  # This argument is no longer needed
        verbose_eval=10,
        callbacks=callbacks
    )
else:
    callbacks = [
        lgbm.log_evaluation(period=10)  # Register the evaluation each 10 intereactions
    ]
    model = lgbm.train(
        params,
        train_data,
        num_boost_round=1000,
        init_model=model,
        start_iteration=start_round,
        valid_sets=[train_data],
        early_stopping_rounds=None,  # This argument is no longer needed
        verbose_eval=10,
        callbacks=callbacks
    )


# Save the trained model
joblib.dump(model, model_path)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 4.394171 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[Li

['lgbm_model.pkl']

## Predictions

In [ ]:
# Perform predictions with the model loaded
y_pred = model.predict(X_test_flat)


# Evaluating

In [ ]:
from sklearn.metrics import classification_report

# Obtener las etiquetas predichas como clases discretas
y_pred_labels = np.argmax(y_pred, axis=1)

# Obtener las etiquetas reales de prueba como clases discretas
y_test_labels = np.argmax(y_test, axis=1)

# Generar el informe de clasificación
report = classification_report(y_test_labels, y_pred_labels)
print(report)

              precision    recall  f1-score   support

           0       0.33      1.00      0.50     16467
           1       0.00      0.00      0.00      1420
           4       0.00      0.00      0.00     15559
           5       0.00      0.00      0.00       833
           6       0.00      0.00      0.00       382
           7       0.00      0.00      0.00      2292
           8       0.00      0.00      0.00        65
           9       0.00      0.00      0.00        22
          10       0.00      0.00      0.00      2331
          11       0.00      0.00      0.00         7
          13       0.00      0.00      0.00       603
          14       0.00      0.00      0.00         8
          15       0.00      0.00      0.00        61
          18       0.00      0.00      0.00        16
          19       0.00      0.00      0.00       378
          21       0.00      0.00      0.00         1
          22       0.00      0.00      0.00        18
          25       0.00    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
'''
from sklearn.preprocessing import MinMaxScaler

# Dimensiones para escalar (2 y 3)
dimensiones = (2, 3)

# Obtener los valores correspondientes a las dimensiones seleccionadas
valores = copia_X[:, :, dimensiones[0], dimensiones[1], :]

# Tomar solo las 8 primeras filas
valores = valores[:, :8]

print(valores.shape)

# Reshape para ajustar a la entrada del escalador
valores_reshaped = valores.reshape(-1, 1)

# Crear un escalador para escalar entre 0 y 1
escalador = MinMaxScaler(feature_range=(0, 1))

# Escalar los valores
valores_escalados = escalador.fit_transform(valores_reshaped)

# Asignar los valores escalados nuevamente al arreglo original
copia_X[:, :, dimensiones[0], dimensiones[1], :8] = valores_escalados.reshape(X.shape[0], copia_X.shape[1], 1, 1, 8).reshape(248548, 9, 8, 1)

print(copia_X.shape)
'''
